[[ 18 219 209 ... 196 239 247]
 [ 87 182  58 ... 114 144 255]
 [213  29  92 ... 166 166  93]
 ...
 [222  53 108 ... 141 213 242]
 [119  89 240 ... 196   4  46]
 [ 73 128  86 ...  38 244 176]]


In [ ]:
from numba import cuda

@cuda.jit
def pixel_inversion(image_matrix):

    for i in range(image_matrix.shape[0]):

        for j in range(image_matrix.shape[1]):
            image_matrix[i, j] = 255 - image_matrix[i, j]

In [ ]:
import numpy as np

image_matrix = np.random.randint(0, 256, (128, 128), dtype=np.uint8)
print(image_matrix)
d_image = cuda.to_device(image_matrix)

[[179  33 161 ... 117  26 223]
 [225 215 108 ...  13 203  53]
 [144 114  20 ... 231 222 127]
 ...
 [ 35  71  53 ... 155 113 137]
 [183  87 240 ... 181 199 184]
 [ 11 133 162 ... 175 145 206]]


In [ ]:

d_flat = cuda.to_device(image_matrix)

block_size = (8,8)

blockspergrid_x = (image_matrix.shape[0] + block_size[0] - 1) // block_size[0]
blockspergrid_y = (image_matrix.shape[1] + block_size[1] - 1) // block_size[1]

blockspergrid = (blockspergrid_x, blockspergrid_y)

pixel_inversion[blockspergrid, block_size](image_matrix)

inverted_image = d_flat.copy_to_host().reshape(128, 128)

print(inverted_image)

ERROR:numba.cuda.cudadrv.driver:Call to cuLinkAddData results in CUDA_ERROR_UNSUPPORTED_PTX_VERSION


LinkerError: [222] Call to cuLinkAddData results in CUDA_ERROR_UNSUPPORTED_PTX_VERSION
ptxas application ptx input, line 9; fatal   : Unsupported .version 8.5; current version is '8.4'

In [ ]:
!pip install numba==0.61.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 22.1 MB/s eta 0:00:00
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: numba
    Found existing installation: numba 0.60.0
    Uninstalling numba-0.60.0:
      Successfully uninstalled numba-0.60.0


In [ ]:
from numba import cuda
import numpy as np
import time

@cuda.jit
def invert_img(image):
    x, y = cuda.grid(2)  # get 2D thread indices
    if x < image.shape[0] and y < image.shape[1]:
        image[x, y] = 255 - image[x, y]


image_matrix = np.random.randint(0, 256, (128, 128), dtype=np.uint8)

d_image = cuda.to_device(image_matrix)

threadsperblock = (8, 8)

blockspergrid_x = (image_matrix.shape[0] + threadsperblock[0] - 1) // threadsperblock[0]
blockspergrid_y = (image_matrix.shape[1] + threadsperblock[1] - 1) // threadsperblock[1]
blockspergrid = (blockspergrid_x, blockspergrid_y)

start = time.time()

invert_img[blockspergrid, threadsperblock](d_image)

inverted = d_image.copy_to_host()

end = time.time()

print("Original:\n", image_matrix)
print("Inverted:\n", inverted)
print("Execution time: \n", end - start)

ERROR:numba.cuda.cudadrv.driver:Call to cuLinkAddData results in CUDA_ERROR_UNSUPPORTED_PTX_VERSION


LinkerError: [222] Call to cuLinkAddData results in CUDA_ERROR_UNSUPPORTED_PTX_VERSION
ptxas application ptx input, line 9; fatal   : Unsupported .version 8.5; current version is '8.4'

In [ ]:
!apt-get install python3.10 python3.10-distutils -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'python3-distutils' instead of 'python3.10-distutils'
python3-distutils is already the newest version (3.10.8-1~22.04).
python3-distutils set to manually installed.
python3.10 is already the newest version (3.10.12-1~22.04.11).
python3.10 set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Thu Sep 25 08:26:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0       

In [ ]:
def auto_config(image_shape, max_threads_per_block):

    rows, cols = image_shape

    candidates = [(32, 32), (32, 16), (16, 32), (16, 16),
                  (32, 8), (8, 32), (8, 16), (16, 8)]

    # Pick the largest valid block size that does not exceed the image size
    for bx, by in candidates:
        if bx <= rows and by <= cols and bx * by <= max_threads_per_block:
            threadsperblock = (bx, by)
            break
    else:
        threadsperblock = (min(16, rows), min(16, cols))

    # Compute grid size
    blockspergrid_x = (rows + threadsperblock[0] - 1) // threadsperblock[0]

    blockspergrid_y = (cols + threadsperblock[1] - 1) // threadsperblock[1]

    blockspergrid = (blockspergrid_x, blockspergrid_y)

    return blockspergrid, threadsperblock

In [ ]:
from numba import cuda
import numpy as np
import time

@cuda.jit
def brighten_img(image):
    x, y = cuda.grid(2)  # get 2D thread indices
    if x < image.shape[0] and y < image.shape[1]:
        image[x, y] = image[x, y] * 1.2


image_matrix = np.random.randint(0, 256, (128, 128), dtype=np.uint8)

d_image = cuda.to_device(image_matrix)

blockspergrid, threadsperblock = auto_config((image_matrix.shape[0], 1024))

start = time.time()

invert_img[blockspergrid, threadsperblock](d_image)

inverted = d_image.copy_to_host()

end = time.time()

print("Original:\n", image_matrix)
print("Inverted:\n", inverted)
print("Execution time: \n", end - start)